<a href="https://colab.research.google.com/github/14vaishaligiri/atom/blob/master/AI%26ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import the pandas libraries
#visualization of data libraeies
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins

plt.rcParams['figure.figsize'] = 10,12

import warnings
warnings.filterwarnings('ignore')


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



#Analysing the present condition in India : How it started in India?
"The first **COVID-19** case was reported on 30th January 2020 when a student arrived **Kerala** from Wuhan. Just in next 2 days, Kerela reported 2 more cases. For almost a month, no new cases were reported in India, however, on 2nd March 2020, five new cases of corona virus were reported in Kerala again and since then the cases have been rising affecting **25** states, till now *(Bihar and Manipur being the most recent)*. Here is a brief timeline of the cases in India.
#Recent COVID-19 updates in India":
In India, from Jan 30 to 3:53pm CEST, 28 June 2020, there have been 528,859 confirmed cases of COVID-19 with 16,095 deaths.
Sikkim on Saturday reported its first +ve COVID-19 case\n",
        "- With over 6,500 fresh cases, the Covid in India rose to 1,25,101 on Saturday morning, with 3,720 fatalities\n",
        "- West Bengal asks Railways not to send migrant trains to State till May 26 in view of Cyclone Amphan\n",
        "- 196 new COVID 19 positive cases were reported in Karnataka on Saturday\n",
        "- Complete lockdown in Bengaluru on Sunday. \n",
        " - Bruhat Bengaluru Mahanagara Palike (BBMP) Commissioner B.H. Anil Kumar said the conditions and restrictions on Sunday will be similar to that under coronavirus lockdown 

#How is AI-ML useful in fighting the COVID-19 pandemic? 
1.Medical resource optimization
2.Ensuring demand planning stability
3.Contact tracing 
4.Situational awareness and critical response analysis

#1.1 Scraping the datasets from the [official Govt. website]
#(https://www.mohfw.gov.in)




In [11]:
#for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# for scraping web contents
from bs4 import BeautifulSoup


      



In [13]:
# get data
# link at which web data recides
link = 'https://www.mohfw.gov.in/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content,"html.parser")



In [15]:
# find the table
# our target table is the last table in the page
# get the table head
# table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)
# get all the rows in table head
# it usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)
# get the table tbody
# it contains the contents
tbody = soup.find_all('tbody')[-1]
# print(tbody)
# get all the rows in table body
# each row is each state's entry
body = tbody.find_all('tr')
# print(body)"

# get the table contents
# container for header rows / column title
head_rows = []
# container for table body / contents
body_rows = []
# loop through the head and append each row to head
for tr in head:
      td = tr.find_all(['th', 'td'])
      row = [i.text for i in td]
      head_rows.append(row)
# print(head_rows)
# loop through the body and append each row to body
for tr in body:
      td = tr.find_all(['th', 'td'])
      row = [i.text for i in td]
      body_rows.append(row)
# print(head_rows)

# save contents in a dataframe
# skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6],
columns=head_rows[0])         
# Drop 'S. No.' column
df_bs.drop('S. No.', axis=1, inplace=True)
# there are 36 states+UT in India
df_bs.head(36)


,Name of State / UT,Active Cases*,Cured/Discharged/Migrated*,Deaths**,Total Confirmed cases*
0,Andaman and Nicobar Islands,29,43,0,72
1,Andhra Pradesh,6648,5480,157,12285
2,Arunachal Pradesh,122,54,1,177
3,Assam,2307,4500,9,6816
4,Bihar,2029,6843,59,8931
5,Chandigarh,87,335,6,428
6,Chhattisgarh,618,1914,13,2545
7,Dadra and Nagar Haveli and Daman and Diu,122,55,0,177
8,Delhi,28329,49301,2558,80188
9,Goa,706,420,2,1128


In [ ]:
# Data Cleaning
# date-time information
#saving a copy of the dataframe
df_India = df_bs.copy()
# today's date
now  = datetime.now()
# format date to month-day-year
df_India['Date'] = now.strftime(\"%m/%d/%Y\") 
# add 'Date' column to dataframe
df_India['Date'] = pd.to_datetime(df_India['Date'], format='%m/%d/%Y')
#df_India.head(36)



In [33]:
# remove extra characters from 'Name of State/UT' column
df_India['Name of State / UT'] = df_India['Name of State / UT'].str.replace('#', '')
df_India['Deaths**'] = df_India['Deaths**'].str.replace('#', '')


# latitude and longitude information
#latitude of the states
lat = {'Delhi':28.7041, 'Haryana':29.0588, 'Kerala':10.8505, 'Rajasthan':27.0238,
        'Telengana':18.1124, 'Uttar Pradesh':26.8467, 'Ladakh':34.2996, 'Tamil Nadu':11.1271,
        'Jammu and Kashmir':33.7782, 'Punjab':31.1471, 'Karnataka':15.3173, 'Maharashtra':19.7515,
        'Andhra Pradesh':15.9129, 'Odisha':20.9517, 'Uttarakhand':30.0668, 'West Bengal':22.9868, 
        'Puducherry': 11.9416, 'Chandigarh': 30.7333, 'Chhattisgarh':21.2787, 'Gujarat': 22.2587, 
        'Himachal Pradesh': 31.1048, 'Madhya Pradesh': 22.9734, 'Bihar': 25.0961, 'Manipur':24.6637, 
        'Mizoram':23.1645, 'Goa': 15.2993, 'Andaman and Nicobar Islands': 11.7401, 'Assam' : 26.2006, 
        'Jharkhand': 23.6102, 'Arunachal Pradesh': 28.2180, 'Tripura': 23.9408, 'Nagaland': 26.1584, 
        'Meghalaya' : 25.4670, 'Dadar Nagar Haveli' : 20.1809, 'Sikkim': 27.5330}
        
#longitude of the states
long = {'Delhi':77.1025, 'Haryana':76.0856, 'Kerala':76.2711, 'Rajasthan':74.2179,
        'Telengana':79.0193, 'Uttar Pradesh':80.9462, 'Ladakh':78.2932, 'Tamil Nadu':78.6569,
        'Jammu and Kashmir':76.5762, 'Punjab':75.3412, 'Karnataka':75.7139, 'Maharashtra':75.7139,
        'Andhra Pradesh':79.7400, 'Odisha':85.0985, 'Uttarakhand':79.0193, 'West Bengal':87.8550, 
        'Puducherry': 79.8083, 'Chandigarh': 76.7794, 'Chhattisgarh':81.8661, 'Gujarat': 71.1924, 
        'Himachal Pradesh': 77.1734, 'Madhya Pradesh': 78.6569, 'Bihar': 85.3131, 'Manipur':93.9063, 
        'Mizoram':92.9376, 'Goa': 74.1240, 'Andaman and Nicobar Islands': 92.6586, 'Assam' : 92.9376, 
        'Jharkhand': 85.2799, 'Arunachal Pradesh': 94.7278, 'Tripura': 91.9882, 'Nagaland': 94.5624,
        'Meghalaya' : 91.3662, 'Dadar Nagar Haveli' : 73.0169, 'Sikkim': 88.5122}
        
# add latitude column based on 'Name of State / UT' column
df_India['Latitude'] = df_India['Name of State / UT'].map(lat)
# add longitude column based on 'Name of State / UT' column
df_India['Longitude'] = df_India['Name of State / UT'].map(long)
df_India.head(36)
    



,Name of State / UT,Active Cases*,Cured/Discharged/Migrated*,Deaths**,Total Confirmed cases*,Latitude,Longitude
0,Andaman and Nicobar Islands,29,43,0,72,11.7401,92.6586
1,Andhra Pradesh,6648,5480,157,12285,15.9129,79.7400
2,Arunachal Pradesh,122,54,1,177,28.2180,94.7278
3,Assam,2307,4500,9,6816,26.2006,92.9376
4,Bihar,2029,6843,59,8931,25.0961,85.3131
5,Chandigarh,87,335,6,428,30.7333,76.7794
6,Chhattisgarh,618,1914,13,2545,21.2787,81.8661
7,Dadra and Nagar Haveli and Daman and Diu,122,55,0,177,NaN,NaN
8,Delhi,28329,49301,2558,80188,28.7041,77.1025
9,Goa,706,420,2,1128,15.2993,74.1240


In [39]:
# rename columns
df_India = df_India.rename(columns={'Cured/Discharged/Migrated' :'Cured/Discharged',
                                    'Total Confirmed cases *': 'Confirmed', 
                                    'Total Confirmed cases ': 'Confirmed',
                                    'Total Confirmed cases* ': 'Confirmed'})

df_India = df_India.rename(columns={'Cured/Discharged':'Cured'})
df_India = df_India.rename(columns={'Name of State / UT':'State/UnionTerritory'})
df_India = df_India.rename(columns={'Name of State / UT':'State/UnionTerritory'})
df_India = df_India.rename(columns=lambda x: re.sub('Total Confirmed cases'  , 
                                                    'Total Confirmed cases',x))
df_India = df_India.rename(columns={'Deaths ( more than 70% cases due to comorbidities )':'Deaths','Deaths**':'Deaths'})

# unique state names
df_India['State/UnionTerritory'].unique()



array(['Andaman and Nicobar Islands', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh',
       'Chhattisgarh', 'Dadra and Nagar Haveli and Daman and Diu',
       'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh',
       'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh',
       'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram',
       'Nagaland', 'Odisha', 'Puducherry', 'Punjab', 'Rajasthan',
       'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttarakhand',
       'Uttar Pradesh', 'West Bengal'], dtype=object)

In [40]:
# number of missing values
df_India.isna().sum()


State/UnionTerritory          0
Active Cases*                 0
Cured/Discharged/Migrated*    0
Deaths                        0
Total Confirmed cases*        0
Latitude                      2
Longitude                     2
dtype: int64

In [41]:
# number of unique values 
df_India.nunique()


State/UnionTerritory          35
Active Cases*                 34
Cured/Discharged/Migrated*    34
Deaths                        26
Total Confirmed cases*        34
Latitude                      33
Longitude                     30
dtype: int64

In [ ]:

# saving data
# file names as year-month-day.csv format
file_name = now.strftime(\"%Y_%m_%d\")+' - COVID-19_India.csv'
# location for saving the file
file_loc = '/content/'
# save file as a scv file
df_India.to_csv(file_loc + file_name, index=False)
# df_India.head(36)
# fix datatype
df_India['Date'] = pd.to_datetime(df_India['Date'])
# rename state/UT names\
df_India['State/UnionTerritory'].replace('Chattisgarh', 'Chhattisgarh', inplace=True)
df_India['State/UnionTerritory'].replace('Pondicherry', 'Puducherry', inplace=True) 
# Final dataframe
df_India.head(36)



In [46]:
# complete data info
df_India.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   State/UnionTerritory        35 non-null     object 
 1   Active Cases*               35 non-null     object 
 2   Cured/Discharged/Migrated*  35 non-null     object 
 3   Deaths                      35 non-null     object 
 4   Total Confirmed cases*      35 non-null     object 
 5   Latitude                    33 non-null     float64
 6   Longitude                   33 non-null     float64
dtypes: float64(2), object(5)
memory usage: 2.0+ KB


In [49]:
# Save as .csv file
# file names as year-month-day.csv format
file_name = now.strftime(\"%Y_%m_%d\")+' - COVID-19_India_preprocessed.csv'
# location for saving the file
file_loc = '/content/'
# save file as a scv file
df_India.to_csv(file_loc + file_name, index=False)


SyntaxError: ignored

In [53]:
#Learn how to read a .csv file by creating a dataframe using pandas
# Reading the datasets
df= pd.read_csv('/content/2020_05_24 - COVID-19_India_preprocessed.csv')
df_india = df.copy()
df

# 1.2 Analysing COVID19 Cases in India
total_cases = df['Confirmed'].sum()
print('Total number of confirmed COVID 2019 cases across India till date (23rd May, 2020):', total_cases)


FileNotFoundError: ignored

In [54]:
#Learn how to highlight your dataframe
df_temp = df.drop(['Latitude', 'Longitude', 'Date', 'index', 'level_0'], axis = 1) 
#Removing Date, Latitude and Longitude and other extra columns
df_temp.style.background_gradient(cmap='Reds')


NameError: ignored